Re-calculate bimodal parameters for each CCN_observation window. (We want NSD1_sum and NSD2_sum for each window)

In [1]:
import numpy as np
import os
import pandas as pd


In [2]:
# load data:
obs_dir = '../input_data'   

bimodal_params = pd.read_csv(os.path.join(obs_dir, 'Bimodal_parameters.csv'))  # Fitted bimodal parameters, which we are recalculating because it does not contain scaled number concentration values.
NSD_params_all = pd.read_csv(os.path.join(obs_dir, 'NSD_PARAMS_SCALED.csv'), parse_dates=['datetime'])  # bimodal parameters scaled to aerosol observations at 10min resolution
CCN_all = pd.read_csv(os.path.join(obs_dir, 'CCN_all.csv'), parse_dates=['datetime','start_time','end_time'])  # observed CCN data (contains CCN-obs Window information (start and end time), 2hr resolution)

In [ ]:
# function to calculate the median absolute deviation (MAD):

def mad(series):
    median = series.median()
    mad = np.median(np.abs(series - median))
    return max(mad, 0.01)  # ensure MAD is not zero

In [4]:
#re-calculate median bimodal parameters for each 2 hr window, with scaled number concentration:

NSD_params = NSD_params_all.copy(deep=True)

tw_start = CCN_all.loc[CCN_all['datetime']>= '2016-08-16 00:00:00'].reset_index(drop=True)['start_time'] # time window start
tw_end = CCN_all.loc[CCN_all['datetime']>= '2016-08-16 00:00:00'].reset_index(drop=True)['end_time'] # time window end
tw_mid = CCN_all.loc[CCN_all['datetime']>= '2016-08-16 00:00:00'].reset_index(drop=True)['datetime'] # time window mid
window_idx = 0

# create mask which identifies which time window parameters belong to:
for i in range(len(tw_start)):
    mask = (
        (NSD_params['datetime'] >= tw_start[i]) &
        (NSD_params['datetime'] < tw_end[i])         
    )
    
    if mask.any():
        NSD_params.loc[mask, 'CCN_window'] = window_idx # index of the time window, used for grouping
        NSD_params.loc[mask, 'datetime'] = tw_mid[i] # assign the mid-point of the time window to the datetime column
        window_idx += 1

# drop rows with NaN in CCN_window column (no data in that time window):
NSD_params = NSD_params.dropna(subset=['CCN_window'])

# drop where the datetime does not match original datetimes in bimodal_params:
NSD_params = NSD_params[NSD_params['datetime'].isin(pd.to_datetime(bimodal_params['datetime']))].reset_index(drop=True)

# THERE IS ONE OBS WINDOW WHERE THE NSD PARAMETERS ARE VERY STRANGE AND RESULT IN UNREALISTIC MASS VALUES:
# here we filter these out:
NSD_params = NSD_params[~((NSD_params['CCN_window'] == 8564.0) & (NSD_params['mode2_d']>100))]

# Take median of parameters within each time window:
NSD_params_windows = NSD_params.groupby('CCN_window').median().copy(deep=True).reset_index(drop=True) # take median of parameters within each time window

# add columns for max/min/median abs. dev. in each time window:
NSD_params_windows['mode1_d_max'] = NSD_params.groupby('CCN_window')['mode1_d'].max().reset_index(drop=True)
NSD_params_windows['mode1_d_min'] = NSD_params.groupby('CCN_window')['mode1_d'].min().reset_index(drop=True)
NSD_params_windows['mode1_d_mad'] = NSD_params.groupby('CCN_window')['mode1_d'].agg(mad).reset_index(drop=True)
NSD_params_windows['mode2_d_max'] = NSD_params.groupby('CCN_window')['mode2_d'].max().reset_index(drop=True)
NSD_params_windows['mode2_d_min'] = NSD_params.groupby('CCN_window')['mode2_d'].min().reset_index(drop=True)
NSD_params_windows['mode2_d_mad'] = NSD_params.groupby('CCN_window')['mode2_d'].agg(mad).reset_index(drop=True)
NSD_params_windows['mode1_NSD_max'] = NSD_params.groupby('CCN_window')['NSD1_sum'].max().reset_index(drop=True)
NSD_params_windows['mode1_NSD_min'] = NSD_params.groupby('CCN_window')['NSD1_sum'].min().reset_index(drop=True)
NSD_params_windows['mode1_NSD_mad'] = NSD_params.groupby('CCN_window')['NSD1_sum'].agg(mad).reset_index(drop=True)
NSD_params_windows['mode2_NSD_max'] = NSD_params.groupby('CCN_window')['NSD2_sum'].max().reset_index(drop=True)
NSD_params_windows['mode2_NSD_min'] = NSD_params.groupby('CCN_window')['NSD2_sum'].min().reset_index(drop=True)
NSD_params_windows['mode2_NSD_mad'] = NSD_params.groupby('CCN_window')['NSD2_sum'].agg(mad).reset_index(drop=True)

# add column for number of measurements in each time window:
NSD_params_windows['n_measurements'] = NSD_params.groupby('CCN_window')['mode1_d'].count()

# create a unique ID for each time window
NSD_params['window_id'] = pd.factorize(NSD_params['CCN_window'])[0]  

In [10]:
NSD_params_windows.loc[NSD_params_windows['mode2_NSD_mad'] == 0]

,datetime,mode1_d,mode1_sigma,mode1_n,mode2_d,mode2_sigma,mode2_n,NSD1_sum,NSD2_sum,mode1_d_max,...,mode2_d_max,mode2_d_min,mode2_d_mad,mode1_NSD_max,mode1_NSD_min,mode1_NSD_mad,mode2_NSD_max,mode2_NSD_min,mode2_NSD_mad,n_measurements
99,2016-08-25 19:00:00,13.765372,1.750000,261.596568,113.426747,1.750000,839.427733,248.743899,775.762152,13.765372,...,113.426747,113.426747,0.0,248.743899,248.743899,0.0,775.762152,775.762152,0.0,NaN
1973,2018-03-30 19:00:00,15.249053,1.750000,4264.047211,83.182413,1.750000,883.716169,3781.114032,873.989737,15.249053,...,83.182413,83.182413,0.0,3781.114032,3781.114032,0.0,873.989737,873.989737,0.0,6.0
3754,2019-03-23 19:00:00,18.007549,1.433333,6565.607955,155.314250,1.766667,248.141125,6816.216972,248.223287,18.007549,...,155.314250,155.314250,0.0,6816.216972,6816.216972,0.0,248.223287,248.223287,0.0,NaN
5130,2019-08-07 09:00:00,69.328069,1.633333,1780.849567,240.256783,1.333333,170.503729,1995.218103,169.416520,69.328069,...,240.256783,240.256783,0.0,1995.218103,1995.218103,0.0,169.416520,169.416520,0.0,NaN
5374,2019-08-29 15:00:00,21.318827,1.500000,1817.034915,102.593727,1.566667,2071.828466,1930.669520,2091.058916,21.318827,...,102.593727,102.593727,0.0,1930.669520,1930.669520,0.0,2091.058916,2091.058916,0.0,10.0
6453,2020-03-14 17:00:00,6.875965,1.750000,2604.472164,111.437410,1.750000,93.249578,2330.670777,97.843779,6.875965,...,111.437410,111.437410,0.0,2330.670777,2330.670777,0.0,97.843779,97.843779,0.0,NaN
6454,2020-03-14 19:00:00,8.740077,1.400000,6242.014337,57.453179,2.066667,244.915984,6185.815042,264.493592,8.740077,...,57.453179,57.453179,0.0,6185.815042,6185.815042,0.0,264.493592,264.493592,0.0,NaN


In [15]:
NSD_params[NSD_params['window_id']==90]

,datetime,mode1_d,mode1_sigma,mode1_n,mode2_d,mode2_sigma,mode2_n,NSD1_sum,NSD2_sum,CCN_window,window_id
773,2016-08-24 17:00:00,37.101714,1.75,1808.308132,195.450420,1.75,149.772899,2133.962257,141.296716,90.0,90
774,2016-08-24 17:00:00,33.818323,1.75,1454.485890,149.289926,1.75,233.422077,1455.017013,223.758200,90.0,90
775,2016-08-24 17:00:00,37.101714,1.75,1480.995668,195.450420,1.75,176.184743,1508.229369,160.285708,90.0,90
776,2016-08-24 17:00:00,37.101714,1.75,1471.295596,170.591083,1.75,193.136299,1464.501672,185.631293,90.0,90
777,2016-08-24 17:00:00,37.101714,1.75,1632.535396,170.591083,1.75,203.411880,1581.968927,194.623570,90.0,90
778,2016-08-24 17:00:00,37.101714,1.75,1469.067817,170.591083,1.75,204.450530,1419.978245,195.107754,90.0,90
779,2016-08-24 17:00:00,37.101714,1.75,1406.039793,170.591083,1.75,216.157442,1399.375283,203.741667,90.0,90
780,2016-08-24 17:00:00,33.818323,1.75,1440.486759,149.289926,1.75,258.637023,1403.176983,244.795422,90.0,90
781,2016-08-24 17:00:00,32.201702,1.75,1551.606246,148.893604,1.75,261.533708,1475.467405,249.442848,90.0,90


In [18]:
np.abs(NSD_params[NSD_params['window_id']==90]['mode1_d'] - NSD_params[NSD_params['window_id']==90]['mode1_d'].median())

773    0.000000
774    3.283391
775    0.000000
776    0.000000
777    0.000000
778    0.000000
779    0.000000
780    3.283391
781    4.900012
Name: mode1_d, dtype: float64

In [21]:
# save to CSV:
NSD_params_windows.to_csv(os.path.join(obs_dir, 'bimodal_params_windows.csv')) 

NSD_params.to_csv(os.path.join(obs_dir, 'NSD_params_withwindows.csv'))  